In [1]:
import pyelsed
import cv2
import numpy as np
import glob
from scipy.spatial.transform import Rotation as R
import os
import matplotlib.pyplot as plt
import json
import open3d as o3d
from skimage.measure import LineModelND, ransac
from sklearn.cluster import KMeans
# from collections import Counter
from joblib import Parallel, delayed
from scipy import stats
params ={
    "perturb_len":6,
}

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


### Replace gt semantic labels with the predicted ones, evaluate accuracy and completeness

In [ ]:
project_root_dir = '/data1/home/lucky/IROS25/SCORE/'
scene_list = ["a1d9da703c","689fec23d7","c173f62b15","69e5939669"]
scene_id = scene_list[0]
mask_folder = project_root_dir+f'/S1_a1d_segmentation_mask/'
# load data with ground truth semantic labels
data_query_path = os.path.join(project_root_dir,f'line_map_extractor/out/{scene_id}/query/{scene_id}_query_data.npy')
data_query = np.load(data_query_path,allow_pickle=True).item()
scene_line_2D_params = data_query['scene_line_2D_params']
scene_line_2D_points = data_query['scene_line_2D_points']
scene_line_2D_semantic_ids = data_query['scene_line_2D_semantic_ids']
scene_line_2D_end_points = data_query['scene_line_2D_end_points']
id_label_dict = data_query['id_label_dict']
id_label_dict[0] = 'unlabeled'
# load predicted semantic masks
mask_files = glob.glob(mask_folder + '*.npy')
mask_files.sort()
###
scene_line_2D_semantic_ids_new={}
total_num_lines = 0
total_num_wrong_predictions = 0
total_num_missing_predictions = 0
wrong_labels = {}
for mask_file in mask_files:
    image_sem_mask = np.load(mask_file)
    base_name=os.path.basename(mask_file).split(".")[0]
    if base_name not in scene_line_2D_points:
        continue
    line_2D_semantic_id = scene_line_2D_semantic_ids[base_name]
    line_2D_semantic_ids_new=[]
    missing_count = 0
    wrong_count = 0
    print("processing",base_name)
    line_2D_points = scene_line_2D_points[base_name]
    num_lines = len(line_2D_points)
    for j in range(num_lines):
        x = line_2D_points[j][0]
        y = line_2D_points[j][1]
        v = np.array([x[0]-x[-1],y[0]-y[-1]])
        v = v/np.linalg.norm(v)
        perturb_list = np.array([0,-1,1])*params['perturb_len']
        old_label = line_2D_semantic_id[j]
        semantic_buffer=[]
        for perturb_l in perturb_list:
            x_p = x + int(perturb_l*v[0])
            y_p = y + int(perturb_l*v[1])
            idx_valid0 = np.logical_and(x_p>=0,x_p<image_sem_mask.shape[1])
            idx_valid1 = np.logical_and(y_p>=0,y_p<image_sem_mask.shape[0])
            idx_valid = np.logical_and(idx_valid0,idx_valid1)
            x_p = x_p[idx_valid].astype(int)
            y_p = y_p[idx_valid].astype(int)
            for i in range(len(x_p)):
                if image_sem_mask[y_p[i],x_p[i]]==0:
                    continue
                semantic_buffer.append(image_sem_mask[y_p[i],x_p[i]])     
        if semantic_buffer==[]:  # no semantic label assigned
            new_label=0
            missing_count+=1
        else:
            new_labels = np.unique(semantic_buffer)
            if old_label not in new_labels: # wrong label assigned
                wrong_count+=1
                new_label = new_labels[0] # take the first label as the new label
            else:
                new_label = old_label
        #
        line_2D_semantic_ids_new.append(new_label)
        if new_label != old_label: # record the wrong prediction
            if (id_label_dict[old_label] + ' ' + id_label_dict[new_label]) not in wrong_labels:
                wrong_labels[id_label_dict[old_label] + ' ' + id_label_dict[new_label]] = 1
            else:
                wrong_labels[id_label_dict[old_label] + ' ' + id_label_dict[new_label]] += 1
            print(f'{base_name},line_{str(j)}, old label:{id_label_dict[old_label]}, new label:{id_label_dict[new_label]}')
    # statistics for this image          
    scene_line_2D_semantic_ids_new[base_name]=line_2D_semantic_ids_new
    total_num_lines += num_lines
    total_num_wrong_predictions += wrong_count
    total_num_missing_predictions += missing_count
print(f'scene {scene_id}, average wrong prediction ratio: {total_num_wrong_predictions/total_num_lines}, average missing prediction ratio: {total_num_missing_predictions/total_num_lines}')

processing frame_000070
frame_000070,line_1, old label:wall, new label:sofa cushion
frame_000070,line_2, old label:floor, new label:sofa cushion
frame_000070,line_3, old label:sofa, new label:sofa cushion
frame_000070,line_4, old label:door, new label:unlabeled
frame_000070,line_5, old label:sofa, new label:sofa cushion
frame_000070,line_6, old label:wall, new label:floor
frame_000070,line_7, old label:sofa, new label:sofa cushion
frame_000070,line_8, old label:wall, new label:unlabeled
frame_000070,line_9, old label:sofa, new label:sofa cushion
frame_000070,line_10, old label:door, new label:unlabeled
frame_000070,line_12, old label:sofa, new label:sofa cushion
processing frame_000150
frame_000150,line_0, old label:door, new label:wall
frame_000150,line_1, old label:door, new label:unlabeled
frame_000150,line_4, old label:door, new label:unlabeled
frame_000150,line_5, old label:floor, new label:door
processing frame_000230
frame_000230,line_0, old label:door, new label:unlabeled
frame

### print wrong or missing prediction pairs
- e.g. ('wall unlabeled', 146): there are 146 2D lines in query images which have semantics 'wall' but are not labeled.

In [3]:
sorted_wrong_labels = []
for key in wrong_labels:
    sorted_wrong_labels.append((key,wrong_labels[key]))
sorted_wrong_labels.sort(key=lambda x:x[1],reverse=True)
for wrong_label in sorted_wrong_labels:
    print(wrong_label)

('wall unlabeled', 146)
('file cabinet table', 88)
('table unlabeled', 87)
('window unlabeled', 78)
('window sill unlabeled', 74)
('monitor unlabeled', 50)
('file cabinet unlabeled', 48)
('door unlabeled', 43)
('storage cabinet door', 35)
('laptop stand laptop', 30)
('storage cabinet unlabeled', 27)
('pipe unlabeled', 26)
('floor unlabeled', 24)
('office chair unlabeled', 22)
('heater unlabeled', 22)
('sofa sofa cushion', 21)
('table keyboard', 21)
('ceiling unlabeled', 20)
('ceiling lamp unlabeled', 19)
('monitor computer tower', 18)
('blinds unlabeled', 18)
('wall file cabinet', 16)
('door wall', 15)
('window sill file cabinet', 15)
('monitor file cabinet', 13)
('window sill table', 13)
('laptop stand unlabeled', 12)
('office chair table', 11)
('whiteboard unlabeled', 10)
('box file cabinet', 9)
('wall table', 9)
('whiteboard wall', 8)
('window sill window', 8)
('wall blinds', 8)
('coat hanger unlabeled', 7)
('monitor laptop stand', 7)
('blinds window', 6)
('floor table', 6)
('floor 

### Output numpy files

In [4]:
data_query_pred = data_query
data_query_pred['scene_line_2D_semantic_ids'] = scene_line_2D_semantic_ids_new
data_query_pred['scene_label_missing_ratio'] = scene_missing_ratio
data_query_pred['scene_wrong_ratio'] = scene_wrong_ratio
np.save(project_root_dir+f'line_map_extractor/out/{scene_id}/query/{scene_id}_query_data_pred.npy',data_query_pred)